## Assignment - 8
In PyQuil, implement the Deutsch-Jozsa algorithm and the Bernstein-Vazirani algorithm.  Write detailed comments in the code about why it works.  Run the programs on the simulator.  Write a report that covers the following three points.

1. Design

    * Present the design of how you implemented the black-box function U_f.  Assess how visually neat and easy to read it is.
    * Present the design for how you prevent the user of U_f from accessing the implementation of U_f.  Assess how well you succeeded.
    * Present the design of how you parameterized the solution in n.
    * Discuss the number of lines and percentage of code that your two programs share.  Assess how well you succeeded in reusing code from one program to the next.


2. Evaluation

    * Discuss your effort to test the two programs and present results from the testing.  Discuss whether different cases of U_f lead to different execution times.
    * What is your experience with scalability as n grows?  Present a diagram that maps n to execution time.

3. Instructions

    * Present a README file that describes how to input the function f, how to run the program, and how to understand the output.
    * Submit three files, one for each program and one with the report.

In [4]:
import numpy as np
N = 2
def f(x):
	# print(x, type(x))
	if(x =="01" or x=="10"):
		return 1
	else:
		return 0
def f_1(x):
	return 1

def f_2(x):
	return 0

def createUf(f):
	indices_dict = {}
	counter = 0
	Uf = np.zeros((2**(N+1),2**(N+1)))
# 	print(Uf)
	bfx = 0

	b = 0
	for i in range(2**N):
		binary_form = str(b) + ('{0:0'+str(N)+'b}').format(i) 
		indices_dict[binary_form] = counter
		counter += 1
		# print(binary_form)

	b = 1
	for i in range(2**N):
		binary_form = str(b) + ('{0:0'+str(N)+'b}').format(i) 
		indices_dict[binary_form] = counter
		counter += 1
		# print(binary_form)

# 	print(indices_dict)
	for key,val in indices_dict.items():
		x = key[0:N]
		fx = str(f(x))
		b = key[N]
		if(b==fx):
			bfx = '0'
		else:
			bfx = '1'
			# print(bfx)
		target = x + bfx
		# print('t',target)
		target_index = indices_dict[target]
		# print(val,target_index)
		Uf[val][target_index] = 1
	return Uf



# Uf = createUf(f)
# print(Uf)

In [5]:
def getBitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

def getFunctionResult(function,x,ancilla):
    fx = function(x)
    newAncilla = str((((int)(ancilla) + fx)%2))
    result = x+newAncilla
    print("\tFunction Input:{} Output:{}".format(x+ancilla,result))
    return getBitVector(result)

def verifyUf(function,n):
    UfMatrix = createUf(function)   
    print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")
    inputs = getAllPossibleNBitStrings(n+1)
    valid = True
    
    for i in inputs:
        inputBitVector = getBitVector(i)
        print("Input BitString {}".format(i))
        print("Input BitVector {}\n".format(inputBitVector))
        
        functionOutput = getFunctionResult(function,i[0:n],i[n])
        UfOutput = np.matmul(UfMatrix,inputBitVector)
        print("\tUf Output {}".format(UfOutput))
        print("\t F Output {}\n".format(functionOutput))
        
        if np.array_equal(UfOutput,functionOutput) is not True:
            print("ERROR FOR {}\n".format(i))
            valid=False
            
    return valid


print("\nVerifying F1 constant\n")
result = verifyUf(f_1,2)
print("\nF_1 constant verification result {}\n".format(result))
print("\nVerifying F2 constant\n")
result = verifyUf(f_2,2)
print("\nF_2 constant verification result {}\n".format(result))

print("\nVerifying F balanced\n")
result = verifyUf(f,2)
print("\nF balanced verification result {}\n".format(result))


Verifying F1 constant


UfMatrix for function 
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]

Input BitString 000
Input BitVector [1 0 0 0 0 0 0 0]

	Function Input:000 Output:001
	Uf Output [0. 1. 0. 0. 0. 0. 0. 0.]
	 F Output [0 1 0 0 0 0 0 0]

Input BitString 100
Input BitVector [0 0 0 0 1 0 0 0]

	Function Input:100 Output:101
	Uf Output [0. 0. 0. 0. 0. 1. 0. 0.]
	 F Output [0 0 0 0 0 1 0 0]

Input BitString 010
Input BitVector [0 0 1 0 0 0 0 0]

	Function Input:010 Output:011
	Uf Output [0. 0. 0. 1. 0. 0. 0. 0.]
	 F Output [0 0 0 1 0 0 0 0]

Input BitString 110
Input BitVector [0 0 0 0 0 0 1 0]

	Function Input:110 Output:111
	Uf Output [0. 0. 0. 0. 0. 0. 0. 1.]
	 F Output [0 0 0 0 0 0 0 1]

Input BitString 001
Input BitVector [0 1 0 0 0 0 0 0]

	Function Input:001 Output:000
	Uf Output [1. 0. 0. 0. 0. 0. 0. 0.]

In [ ]:
getBitVector("000")

In [10]:
from pyquil import Program
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit
from pyquil.quil import DefGate

    
    
def runMainCircuit(function,n,nTrials):
    p = Program()
    
    n = n+1
    qc_name = "{}q-qvm".format(n)
    # Get our QuantumComputer instance, with a Quantum Virutal Machine (QVM) backend
    qc = get_qc(qc_name)

    #setting last qubit to 1
    p += X(n-1)
    
    #adding Hadamard gates to all qubits
    for i in range(0,n):
        p += H(i)


    UfMatrix = createUf(function)
    GateName = "UF_GATE"

    uf_gate_definition = DefGate(GateName, UfMatrix)
    qubits = [unpack_qubit(i) for i in range(0,n)]

    #adding Uf gate
    p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))

    for i in range(0,n-1):
        p += H(i)

    print(p)
        
    results = qc.run_and_measure(p, trials=nTrials)
    return results

In [185]:
runMainCircuit(f,2)
runMainCircuit(f_1,2)

DEFGATE UF_GATE:
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0

X 2
H 0
H 1
H 2
UF_GATE 0 1 2
H 0
H 1

{0: array([0, 0]), 1: array([0, 0]), 2: array([1, 1])}
DEFGATE UF_GATE:
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0

X 2
H 0
H 1
H 2
UF_GATE 0 1 2
H 0
H 1

{0: array([0, 0]), 1: array([0, 0]), 2: array([0, 1])}


### Deutsch-Jozsa

In [8]:
nTrials = 1

def isBalancedOrConstant(function,nQubits):
    results = runMainCircuit(function,nQubits,nTrials)
    constant = True
    for i in range(0,nQubits):
        if results[i][0] != 0:
            constant = False
            break
    
    if constant:
        return "CONSTANT"
    else:
        return "BALANCED"

In [14]:
print(isBalancedOrConstant(f,2))
print(isBalancedOrConstant(f_1,2))
print(isBalancedOrConstant(f_2,2))

DEFGATE UF_GATE:
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0

X 2
H 0
H 1
H 2
UF_GATE 0 1 2
H 0
H 1

BALANCED
DEFGATE UF_GATE:
    0, 1.0, 0, 0, 0, 0, 0, 0
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0
    0, 0, 0, 0, 0, 0, 1.0, 0

X 2
H 0
H 1
H 2
UF_GATE 0 1 2
H 0
H 1

CONSTANT
DEFGATE UF_GATE:
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0

X 2
H 0
H 1
H 2
UF_GATE 0 1 2
H 0
H 1

CONSTANT


### Bernstein-Vazirani

In [ ]:
def constructFunction(function,nQubits):
    b = function("0"*nQubits)
    a = ""
    for i in range(0,nQubits):
        a+=str(results[i][0])
    return a,b